In [83]:
import numpy as np
import pandas as pd
import os
import re
from wordfreq import get_frequency_dict


### Pdf reading

In [41]:
%pip install --upgrade pdfplumber pdfminer -q

import pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [42]:
def _convert_pdf_to_text(path):
    """Convert the PDF to text and store it in the self.content
    attribute.
    """
    text = []
    pdf = pdfplumber.open(path)
    for page in range(len(pdf.pages)):

        page_obj = pdf.pages[page]

        text.append(
            page_obj.extract_text(
                x_tolerance=1, x_tolerance_ratio=None, y_tolerance=1, layout=False, x_density=7.25, y_density=13, line_dir_render=None,keep_blank_chars=True,use_text_flow=True, char_dir_render=None, expand_ligatures=True
            )
        )
    text = "\n".join(text)
    
    return text

In [96]:

def process_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        text =  _convert_pdf_to_text(file)

        

        def pa(text):
            pattern_abstract = r'\s*a\s*b\s*s\s*t\s*r\s*a\s*c\s*t\s*'
            matches1 = re.findall(pattern_abstract, text)
            if matches1:
                match = ''.join(matches1[0])
                pattern_abstract = re.compile(pattern_abstract)
                return pattern_abstract.sub('\nabstract\n', text)
            else:
                return text

        def pi(text):
            pattern_intro = r'i\s*n\s*t\s*r\s*o\s*d\s*u\s*c\s*t\s*i\s*o\s*n'
            matches1 = re.findall(pattern_intro, text)
            if matches1:
                match = ''.join(matches1[0])
                pattern_intro = re.compile(pattern_intro)
                return pattern_intro.sub('\nintroduction\n', text)
            else:
                return text

        def pr(text):
            pattern_intro = r'r\s*e\s*f\s*e\s*r\s*e\s*n\s*c\s*e\s*s'
            matches1 = re.findall(pattern_intro, text)
            if matches1:
                match = ''.join(matches1[0])
                pattern_intro = re.compile(pattern_intro)
                return pattern_intro.sub('\nreferences\n', text)
            else:
                return text
            
        text = pa(text)
        text = pi(text)
        text = pr(text)
        # print(text.lower())
        text = [[text]]
        df = pd.DataFrame(text, columns=['raw_text'])

        def find_text_by_largest_font(page):
            largest_font_size = 0
            largest_font_chars = []

            for char in page.chars:
                size = char["size"]
                if size > largest_font_size:
                    largest_font_size = size
                    largest_font_chars = [char]
                elif size == largest_font_size:
                    largest_font_chars.append(char)

            return largest_font_chars

        with pdfplumber.open(file) as pdf:
            first_page = pdf.pages[0]
            largest_font_chars = find_text_by_largest_font(first_page)
            largest_font_text = ''.join(char['text'] for char in largest_font_chars)
        df['Title'] = largest_font_text


        word_prob = get_frequency_dict(lang='en', wordlist='large')
        max_word_len = max(map(len, word_prob))  # 34

        # def str_slice(text):
        def viterbi_segment(text, debug=False):
            probs, lasts = [1.0], [0]
            for i in range(1, len(text) + 1):
                new_probs = []
                for j in range(max(0, i - max_word_len), i):
                    substring = text[j:i]
                    length_reward = np.exp(len(substring))
                    freq = word_prob.get(substring, 0) * length_reward
                    compounded_prob = probs[j] * freq
                    new_probs.append((compounded_prob, j))
                    
                    if debug:
                        print(f'[{j}:{i}] = "{text[lasts[j]:j]} & {substring}" = ({probs[j]:.8f} & {freq:.8f}) = {compounded_prob:.8f}')

                prob_k, k = max(new_probs)  # max of a touple is the max across the first elements, which is the max of the compounded probabilities
                probs.append(prob_k)
                lasts.append(k)

                if debug:
                    print(f'i = {i}, prob_k = {prob_k:.8f}, k = {k}, ({text[k:i]})\n')


            # when text is a word that doesn't exist, the algorithm breaks it into individual letters.
            # in that case, return the original word instead
            if len(set(lasts)) == len(text):
                return text

            words = []
            k = len(text)
            while 0 < k:
                word = text[lasts[k]:k]
                words.append(word)
                k = lasts[k]
            words.reverse()
            return ' '.join(words)

        def split_message(message):
            new_message = ' '.join(viterbi_segment(wordmash, debug=False) for wordmash in message.split())
            return new_message

        def raw(text):
            if 'introduction' in text:
                new = text.split('introduction')
                return new[1] 
            else:
                return ''

        def header(text):
            if 'introduction' in text:
                new = text.split('introduction')
                return new[0]   
            else:
                return ''
            
        def abstract(text):
            new = text.split('abstract')
            if len(new) > 1:
                return new[1]
            else:
                return ''  
            
            

        def pc(data_text):
            data_text = data_text.split('\"' and "," and ":" and '\”' and '\“')
            text = ""
            for i in data_text:
                text = text + i
            print(len(text))

            text = ""

            for i in data_text:
                text = text + i
    
            return text
        
        mail_pattern = re.compile(r'\b\S+@\S+\.\S+\.?\S*\b')
        def remove_mail(text):
            return mail_pattern.sub('', text)
        
        url_pattern = r'https?://(?:www\.)?[^ ]+|www[^ ]+'

        def add_urls(text):
            httplinks = re.findall(url_pattern,text)
            return httplinks

        url_pattern = re.compile(url_pattern)
        
        def remove_urls(text):
            return url_pattern.sub('', text)
        
        def ref(text):
            if('references' in text):
                terms = text.split('references')
                return terms[1]
            else:
                return ''
        def wref(text):
            if('references' in text):
                terms = text.split('references')
                return terms[0]
            else:
                return text
            
        number_pattern = re.compile(r'\b(?<!\[)(-?\d+(?:\.\d+)?)\b(?![^\[]*\])')

        def remove_number(text):
            return number_pattern.sub('', text)


        def sen_seg(data_text):
            sentences = []

            sentence = ''

            # Iterate through the text
            for i in range(len(data_text)):
                char = data_text[i]
                # Append character to the current sentence
                sentence += char
                if char == '.':
                    # Check if the character before and after '.' are both numbers
                    if i > 0 and i < len(data_text) - 1 and data_text[i-1].isdigit() and data_text[i+1].isdigit():
                        continue  # Skip adding '.' to the sentence
                    else:
                        sentences.append(sentence.strip())
                        sentence = ''

            return sentences
        
        def checker(sentences):
            new = []
            for s in sentences:
                if(len(s) > 30):
                    new.append(s)
            return new

        
        def merge_sen(sentences):
            separator = " "
            clean = separator.join(sentences)
            return clean

        df['Title'] = df['Title'].apply(split_message)
        df['raw_text'] = df['raw_text'].apply(raw)
        df['header'] = df['raw_text'].apply(header)
        df['abstract'] = df['header'].apply(abstract)
        df['pc_removal'] = df['raw_text'].apply(pc)
        df['wmail'] = df['pc_removal'].apply(remove_mail)
        df['urls'] = df['wmail'].apply(add_urls)
        df['wurl'] = df['wmail'].apply(remove_urls)
        df['refs'] = df['wurl'].apply(ref)
        df['wrefs'] = df['wurl'].apply(wref)
        df['wnumbers'] = df['wrefs']
        df['sentences'] = df['wnumbers'].apply(sen_seg)
        df['sentences'] = df['sentences'].apply(checker)
        df['clean'] = df['sentences'].apply(merge_sen)

    return df



def process_directory(directory):
    combined_df = pd.DataFrame() 

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pdf'):
                pdf_path = os.path.join(root, file)
                df = process_pdf(pdf_path)  
                combined_df = pd.concat([combined_df, df], ignore_index=True)

        for dir in dirs:
            subdir = os.path.join(root, dir)
            process_directory(subdir)

    return combined_df

pdf_directory = './../Createdb/docs'

if os.path.isdir(pdf_directory):
    combined_df = process_directory(pdf_directory)
    # Check if the combined DataFrame is not empty
    if not combined_df.empty:
        # Export the combined DataFrame to CSV
        combined_df.to_csv("combined_data.csv", index=False)
    else:
        print("Combined DataFrame is empty.")
else:
    if pdf_directory.endswith('.pdf'):
        df = process_pdf(pdf_directory)  
        # Check if the DataFrame is not empty
        if not df.empty:
            # Export the DataFrame to CSV
            df.to_csv("combined_data.csv", index=False)
        else:
            print("DataFrame is empty.")
    else:
        print("Invalid path. Please provide a directory or a PDF file.")

KeyError: 'Raw_text'

### replacing topics

In [47]:
import re

def pa(text):
    # text = "abstract  abstract  abstract"
    pattern_abstract = r'\s*a\s*b\s*s\s*t\s*r\s*a\s*c\s*t\s*'
    matches1 = re.findall(pattern_abstract,text)
    print(matches1)
    match = ''.join(matches1[0])
    print(match)
    pattern_abstract = re.compile(pattern_abstract)
    return pattern_abstract.sub('\nabstract\n',text)
    # matches1 = re.findall(pattern_abstract, text)
    # text.replace(match,'abstract')
def pi(text):
    # text = "abstract  abstract  abstract"
    pattern_intro = r'i\s*n\s*t\s*r\s*o\s*d\s*u\s*c\s*t\s*i\s*o\s*n'
    matches1 = re.findall(pattern_intro,text)
    print(matches1)
    match = ''.join(matches1[0])
    print(match)
    pattern_intro = re.compile(pattern_intro)
    return pattern_intro.sub('\nintroduction\n',text)
    # matches1 = re.findall(pattern_abstract, text)
    # text.replace(match,'abstract')
def pr(text):
    # text = "abstract  abstract  abstract"
    pattern_intro = r'r\s*e\s*f\s*e\s*r\s*e\s*n\s*c\s*e\s*s'
    matches1 = re.findall(pattern_intro,text)
    if len(matches1) == 0:
        return text 
    print(matches1)
    match = ''.join(matches1[0])
    print(match)
    pattern_intro = re.compile(pattern_intro)
    return pattern_intro.sub('\nreferences\n',text)
    # matches1 = re.findall(pattern_abstract, text)
    # text.replace(match,'abstract')
text = pa(text)
text = pi(text)
text = pr(text)

['\na b s t r a c t\n']

a b s t r a c t

['introduction']
introduction
['references']
references


In [48]:
data = [[text]]

In [49]:
df = pd.DataFrame(data, columns=['raw_text'])

### adding title from first page

In [50]:
# may not be accurate 
# we are assuming that the text with the largest font is the title


import pdfplumber

def find_text_by_largest_font(page):
    largest_font_size = 0
    largest_font_chars = []

    for char in page.chars:
        size = char["size"]
        if size > largest_font_size:
            largest_font_size = size
            largest_font_chars = [char]
        elif size == largest_font_size:
            largest_font_chars.append(char)
    # for char in page.chars:
    #     size = char["size"]
    #     if size == largest_font_size or char == " ":
    #         largest_font_chars.append(char)

    return largest_font_chars


with pdfplumber.open(path) as pdf:
    first_page = pdf.pages[0]
    largest_font_chars = find_text_by_largest_font(first_page)

    largest_font_text = ''.join(char['text'] for char in largest_font_chars)

    # print("-----")
    # print(largest_font_text)
    # print("-----")
df['Title'] = largest_font_text

In [51]:
df

,raw_text,Title
0,journal of constructional steel research 65 (2...,JournalofConstructionalSteelResearch


### String Slicing for Title

In [52]:
pip install wordfreq

Note: you may need to restart the kernel to use updated packages.


In [53]:
# may or may not work depending on the dictionary



import numpy as np
from wordfreq import get_frequency_dict

word_prob = get_frequency_dict(lang='en', wordlist='large')
max_word_len = max(map(len, word_prob))  # 34

# def str_slice(text):
def viterbi_segment(text, debug=False):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        new_probs = []
        for j in range(max(0, i - max_word_len), i):
            substring = text[j:i]
            length_reward = np.exp(len(substring))
            freq = word_prob.get(substring, 0) * length_reward
            compounded_prob = probs[j] * freq
            new_probs.append((compounded_prob, j))
            
            if debug:
                print(f'[{j}:{i}] = "{text[lasts[j]:j]} & {substring}" = ({probs[j]:.8f} & {freq:.8f}) = {compounded_prob:.8f}')

        prob_k, k = max(new_probs)  # max of a touple is the max across the first elements, which is the max of the compounded probabilities
        probs.append(prob_k)
        lasts.append(k)

        if debug:
            print(f'i = {i}, prob_k = {prob_k:.8f}, k = {k}, ({text[k:i]})\n')


    # when text is a word that doesn't exist, the algorithm breaks it into individual letters.
    # in that case, return the original word instead
    if len(set(lasts)) == len(text):
        return text

    words = []
    k = len(text)
    while 0 < k:
        word = text[lasts[k]:k]
        words.append(word)
        k = lasts[k]
    words.reverse()
    return ' '.join(words)

def split_message(message):
    new_message = ' '.join(viterbi_segment(wordmash, debug=False) for wordmash in message.split())
    return new_message

### header

In [54]:
def raw(text):
    if 'introduction' in text:
        new = text.split('introduction')
        return new[1] 
    else:
        return ''

def header(text):
    if 'introduction' in text:
        new = text.split('introduction')
        return new[0]   
    else:
        return ''

In [55]:
import re
def abstract(text):

    new = text.split('abstract')
    return new[1]

### Punctuation removal

In [56]:
import string
import re 
# import nltk
# from nltk.corpus import stopwords



def pc(data_text):
    data_text = data_text.split('\"' and "," and ":" and '\”' and '\“')
    text = ""
    for i in data_text:
        text = text + i
    print(len(text))

    text = ""

    for i in data_text:
        text = text + i
    
    return text



### mail

In [57]:
# mail
mail_pattern = re.compile(r'\b\S+@\S+\.\S+\.?\S*\b')
def remove_mail(text):
    return mail_pattern.sub('', text)

# df['without_mail'] = df['without_refs'].apply(remove_mail)

### url

In [58]:
# #url
url_pattern = r'https?://(?:www\.)?[^ ]+|www[^ ]+'
def add_urls(text):
    httplinks = re.findall(url_pattern,text)
    return httplinks

url_pattern = re.compile(url_pattern)
def remove_urls(text):
    return url_pattern.sub('', text)



In [59]:
# df['urls'][0]

### 

### split refrences

In [60]:
def ref(text):
    if('references' in text):
        terms = text.split('references')
        return terms[1]
    else:
        return ''
def wref(text):
    if('references' in text):
        terms = text.split('references')
        return terms[0]
    else:
        return text


In [61]:
# print(df['refs'][0])

### numbers

In [62]:
import re

number_pattern = re.compile(r'\b(?<!\[)(-?\d+(?:\.\d+)?)\b(?![^\[]*\])')

def remove_number(text):
    return number_pattern.sub('', text)

# Assuming df['without_url'] is your DataFrame column containing text
# df['wnumbers'] = df['wrefs'].apply(remove_number)


### sentence segmentation

In [63]:
def sen_seg(data_text):
    sentences = []

    sentence = ''

    # Iterate through the text
    for i in range(len(data_text)):
        char = data_text[i]
        # Append character to the current sentence
        sentence += char
        if char == '.':
            # Check if the character before and after '.' are both numbers
            if i > 0 and i < len(data_text) - 1 and data_text[i-1].isdigit() and data_text[i+1].isdigit():
                continue  # Skip adding '.' to the sentence
            else:
                sentences.append(sentence.strip())
                sentence = ''

    return sentences

# Assuming df['wnumbers'] contains the text data



# Assuming df['wnumbers'] contains the text data
# sen_seg('fjdkfj.fdsfds.s 3.4 dfds.dsa')


In [64]:
# print(len(df['sentences'][0]))

In [65]:
def checker(sentences):
    new = []
    for s in sentences:
        if(len(s) > 30):
            new.append(s)

    return new



# selecting sentences with more characters than 30

### merging sentences

In [66]:
def merge_sen(sentences):
    separator = " "
    clean = separator.join(sentences)
    return clean


In [67]:



df['Title'] = df['Title'].apply(split_message)
df['header'] = df['raw_text'].apply(header)
df['raw_text'] = df['raw_text'].apply(raw)
df['abstract'] = df['header'].apply(abstract)
df['pc_removal'] = df['raw_text'].apply(pc)
df['wmail'] = df['pc_removal'].apply(remove_mail)
df['urls'] = df['wmail'].apply(add_urls)
df['wurl'] = df['wmail'].apply(remove_urls)
df['refs'] = df['wurl'].apply(ref)
df['wrefs'] = df['wurl'].apply(wref)
df['wnumbers'] = df['wrefs']
df['sentences'] = df['wnumbers'].apply(sen_seg)
df['sentences'] = df['sentences'].apply(checker)
df['clean'] = df['sentences'].apply(merge_sen)

36210


In [86]:
df.to_csv('preProcessed.csv')